## High on Cloud Wine: Predicting the Quality of Wine
From seed to wine, the process of making the said alcohol is said to take up to 5 years before the liquid is bottled and sold to the market for consuming. Moreoever, the sheer level of detail that vintners, or winemakers, need to pay attention to is immense due to the complexities of the taste of the alcohol. A lot of factors goes into this including the type of fruits invovled, the temperature that the fruits were grown in and where the barrels for fermentation were kept, the alcohol content in the drink and a lot more ("Wine Production", 2018). This is why it is important to figure out the quality of wine, not only for the producers to assure the quality of their products, but also for consumers as well to decide which wine they should try out as a beginner trying to get into alcoholic drinks, and for sommeliers as well to decide which ones to pair with their meal/s.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

## Red Wine Quality Dataset
The [Wine Quality Dataset](https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009) (titled **`winequality-red.csv`**) consists of 1600 red and white wine variants of the "Vinho Verde" wine from Portugal. The data was retrieved from a study conducted by Cortez, P. et al. and was uploaded and shared to Kaggle by the staff of **UCI Machine Learning Repository** under the same name. Each row available in the dataset consists of values coming from physiochemical and sensory variables. Information about the producers, grape type and selling prices are said to be unavailable due to privacy and logistic reasons.

In [2]:
main_df = pd.read_csv("./winequality-red.csv")
main_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Below is the list of features, their descriptions and their label:
- **`fixed acidity`**: Acids that do not evaporate quickly (g(tartaric acid)/dm^3).
- **`volatile acidity`**: Acetic acid that gives a vinegary taste when too much is added (g(acetic acid)/dm^3).
- **`citric acid	`**: Adds freshness to the flavor of the wine (g/dm^3).
- **`residual sugar	`**: Amount of sugar after fermentation (g/dm^3).
- **`chlorides`**: Amount of salt (g(sodium chloride)/dm^3).
- **`free sulfur dioxide`**: Amount of free form sulfure dioxide used to prevent microbial growth and oxidation (mg/dm^3).
- **`total sulfur dioxide`**: Amount of free and bound forms of sulfur dioxide (mg/dm^3).
- **`density`**: Density of water (grams/cm^3).
- **`pH`**: How acidic (0) or basic (14) the wine is.
- **`sulphates`**: An additive that contributes to sulfur dioxide gas (g(potassium sulphate)/dm^3).
- **`alcohol`**: Alcohol content percentage (vol.%).
- **`quality`**: Quality of the wine with 0 being the lowest and 10 being the highest.

## References <br>
Apallas, A. K. (2016, July 6). The Life Cycle of a Wine Grape: From Planting to Harvest to Bottle. Wine Cooler Direct. https://learn.winecoolerdirect.com/life-cycle-of-a-wine-grape/<br>
Cortez, P., Cerdeira, A., Almeida, F., Matos, T., & Reis, J. (2009). Modeling wine preferences by data mining from physicochemical properties. Decision Support Systems, 47(4), 547-553. https://doi.org/10.1016/j.dss.2009.05.016<br>
WINE PRODUCTION: BOTTLING QUALITY CONTROL. (2018, September 18). Gravity Wine House. https://gravitywinehouse.com/blog/wine-production-bottling-quality-control-plan/